In [2]:
#import joblib
import pandas as pd
from datetime import datetime
import numpy as np
#import duckdb
import flet
#import pyarrow
import tinydb

In [3]:
JGB_rates = pd.read_csv('https://www.mof.go.jp/jgbs/reference/interest_rate/data/jgbcm_all.csv', encoding='Shift-JIS', header=1)

In [4]:
JGB_rates.head()

,基準日,1年,2年,3年,4年,5年,6年,7年,8年,9年,10年,15年,20年,25年,30年,40年
0,S49.9.24,10.327,9.362,8.83,8.515,8.348,8.290,8.24,8.121,8.127,-,-,-,-,-,-
1,S49.9.25,10.333,9.364,8.831,8.516,8.348,8.290,8.24,8.121,8.127,-,-,-,-,-,-
2,S49.9.26,10.34,9.366,8.832,8.516,8.348,8.290,8.24,8.122,8.128,-,-,-,-,-,-
3,S49.9.27,10.347,9.367,8.833,8.517,8.349,8.290,8.24,8.122,8.128,-,-,-,-,-,-
4,S49.9.28,10.354,9.369,8.834,8.518,8.349,8.291,8.24,8.122,8.129,-,-,-,-,-,-


In [5]:
def dateConv(date):

    wareki = {
            "R":2018,
            "H":1988,
            "S":1925
            }
  
    date_els = date.split('.', 2)
    year_num = int(date_els[0][1:])
    era = date_els[0][0]
 
    year = year_num + wareki[era]

    month =int(date_els[1])
    day = int(date_els[2])

    dt = datetime(year, month, day)

    return dt



In [6]:
JGB_rates["基準日"] = JGB_rates['基準日'].apply(dateConv)
JGB_rates = JGB_rates.set_index('基準日')
JGB_rates.replace('-', np.nan, inplace=True)

In [7]:
JGB_rates.head()

,1年,2年,3年,4年,5年,6年,7年,8年,9年,10年,15年,20年,25年,30年,40年
基準日,,,,,,,,,,,,,,,
1974-09-24,10.327,9.362,8.83,8.515,8.348,8.290,8.24,8.121,8.127,NaN,NaN,NaN,NaN,NaN,NaN
1974-09-25,10.333,9.364,8.831,8.516,8.348,8.290,8.24,8.121,8.127,NaN,NaN,NaN,NaN,NaN,NaN
1974-09-26,10.34,9.366,8.832,8.516,8.348,8.290,8.24,8.122,8.128,NaN,NaN,NaN,NaN,NaN,NaN
1974-09-27,10.347,9.367,8.833,8.517,8.349,8.290,8.24,8.122,8.128,NaN,NaN,NaN,NaN,NaN,NaN
1974-09-28,10.354,9.369,8.834,8.518,8.349,8.291,8.24,8.122,8.129,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
JGB_rates.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12579 entries, 1974-09-24 to 2023-09-29
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1年      11931 non-null  object 
 1   2年      12218 non-null  object 
 2   3年      12503 non-null  object 
 3   4年      12579 non-null  float64
 4   5年      12579 non-null  float64
 5   6年      12579 non-null  float64
 6   7年      12579 non-null  float64
 7   8年      12579 non-null  float64
 8   9年      12579 non-null  float64
 9   10年     9218 non-null   object 
 10  15年     7877 non-null   object 
 11  20年     9105 non-null   object 
 12  25年     4784 non-null   object 
 13  30年     5902 non-null   object 
 14  40年     3890 non-null   object 
dtypes: float64(6), object(9)
memory usage: 1.5+ MB


In [9]:
JGB_rates['5年'].resample('Y').mean()

基準日
1974-12-31    8.377000
1975-12-31    8.483218
1976-12-31    8.735269
1977-12-31    7.500503
1978-12-31    5.929144
1979-12-31    7.424343
1980-12-31    8.638351
1981-12-31    8.210207
1982-12-31    8.126912
1983-12-31    7.693021
1984-12-31    6.959432
1985-12-31    6.359263
1986-12-31    5.153240
1987-12-31    4.267631
1988-12-31    4.180077
1989-12-31    5.005743
1990-12-31    7.051776
1991-12-31    6.474069
1992-12-31    4.856466
1993-12-31    3.606285
1994-12-31    3.673360
1995-12-31    2.608570
1996-12-31    2.208781
1997-12-31    1.546653
1998-12-31    1.018368
1999-12-31    0.983000
2000-12-31    1.101435
2001-12-31    0.540703
2002-12-31    0.441244
2003-12-31    0.409922
2004-12-31    0.666313
2005-12-31    0.628220
2006-12-31    1.229774
2007-12-31    1.236090
2008-12-31    1.016457
2009-12-31    0.702761
2010-12-31    0.419980
2011-12-31    0.427437
2012-12-31    0.246508
2013-12-31    0.235371
2014-12-31    0.162418
2015-12-31    0.078107
2016-12-31   -0.187139
2017-12

In [10]:
JGB_rate_5_30 = JGB_rates[['5年', '10年', '15年', '20年', '25年', '30年']].dropna()
JGB_rate_5_30 = JGB_rate_5_30.astype(float, errors='raise')

In [11]:
JGB_rate_5_30_12M = JGB_rate_5_30.resample('M').mean().tail(12)
JGB_rate_5_30_12M

,5年,10年,15年,20年,25年,30年
基準日,,,,,,
2022-10-31,0.090850,0.257150,0.769350,1.081450,1.298250,1.456100
2022-11-30,0.087100,0.265300,0.786750,1.104550,1.314900,1.480100
2022-12-31,0.178273,0.357045,0.873182,1.164773,1.351409,1.485909
2023-01-31,0.235947,0.479053,1.054789,1.327947,1.489789,1.575895
2023-02-28,0.214895,0.520053,1.047158,1.310474,1.444000,1.508579
2023-03-31,0.150727,0.402455,0.860500,1.135818,1.265727,1.353545
2023-04-30,0.160650,0.466450,0.850750,1.105800,1.241600,1.335400
2023-05-31,0.115050,0.419100,0.761300,1.020850,1.165400,1.265000
2023-06-30,0.089818,0.423455,0.753636,1.010091,1.152227,1.253227


In [15]:
JGB_rate_5_30_12M_dic = JGB_rate_5_30_12M.mean().to_dict()

In [16]:
JGB_rate_5_30_12M_dic.values()

dict_values([0.1445357057416268, 0.41357505980861237, 0.8565380382775118, 1.1341570574162678, 1.2976279106858053, 1.407779246411483])

In [56]:
JGB_rate_5_30_12M_sr = JGB_rate_5_30_12M.mean()
JGB_rate_5_30_12M_df = pd.DataFrame(JGB_rate_5_30_12M_sr, columns=['利率'])
JGB_rate_5_30_12M_df = JGB_rate_5_30_12M_df.transpose()

In [1]:
from tinydb import TinyDB, Query

In [2]:
con = TinyDB('res_02_db.json')

In [3]:
con2 = TinyDB("selected_res.json")
dtime = con2.all()[0]['selected_datetime']
con2.close()
items = Query()
selected_results_dict = con.search(items.datetime == dtime)[0]

In [4]:
selected_results_dict

{'LCC_net_expense': '6347.675',
 'PSC_net_expense_const_kk': '3976.565',
 'PSC_net_expense_ijikanri_kk': '573.851',
 'ijikanri_years': 14,
 'discount_rate': '2.812',
 'rakusatsu_ritsu': 0.95,
 'PSC': '4503.101',
 'LCC': '5055.745',
 'VFM': '-552.643',
 'VFM_percent': '-12.273',
 'mgmt_type': '都道府県',
 'proj_ctgry': 'サービス購入型',
 'proj_type': 'BTO',
 'proj_years': 16,
 'const_years': 2,
 'kijun_kinri': 0.8565380382775118,
 'chisai_kinri': 0.75,
 'zei_modori': 5.78,
 'lg_spread': 1.5,
 'zei_total': 41.98,
 'growth': 0.0,
 'kitai_bukka': 1.9554642942583733,
 'shisetsu_seibi': 6593.5,
 'ijikanri_unnei': 46.0,
 'reduc_shisetsu': 10.0,
 'reduc_ijikanri': 10.0,
 'pre_kyoukouka': False,
 'kisai_jutou': 75.0,
 'kisai_koufu': 30.0,
 'zeimae_rieki': 8.5,
 'SPC_keihi': 15.0,
 'hojo': 50.0,
 'datetime': '2023-10-12 20:15:59',
 'user_id': '01HCHSKESHV3J0577EQ9DPR88W',
 'calc_id': '02uZttXX82aaVTLZ6jOMHe',
 'PSC_present_value': {'0': 2045.8106638693582,
  '1': 1988.2824202820839,
  '2': 39.8682409820108

0.1445357057416268